In [281]:
import pandas as pd
import numpy as np

In [282]:
pd.options.display.max_columns= 1000

In [283]:
utility = pd.read_csv('E:/CR2/Repos/TNC-Demand-Model-Southeast/Cleaned_Inputs/Intermediate/pred_trips.csv')

In [284]:
# process acs s1903 data to get column for median income per census tract

In [285]:
median_inc = pd.read_csv('E:/CR2/Repos/TNC-Demand-Model-Southeast/Raw_Inputs/ACS/S1903/S1903 Data.csv')

In [286]:
median_inc[['TRACT', 'COUNTY', 'STATE']] = median_inc.NAME.str.split(',', expand=True)

In [287]:
median_inc = median_inc[median_inc['COUNTY'] == ' Fayette County']

In [288]:
median_inc['CENSUS_TRACT'] = median_inc.apply(lambda row: row['GEO_ID'][9:], axis = 1)

In [289]:
median_inc = median_inc.rename(columns={'S1903_C03_001E': 'MEDIAN_INC'})

In [290]:
median_inc = median_inc[['CENSUS_TRACT', 'MEDIAN_INC']]

In [291]:
# merge utility with median on origin

In [292]:
utility = utility.merge(median_inc.astype(float), how='left', left_on='ORIGIN', right_on='CENSUS_TRACT')

In [293]:
utility.rename(columns={'MEDIAN_INC': 'ORIGIN_MEDIAN_INC'}, inplace=True)
utility.drop(columns='CENSUS_TRACT', inplace=True)

In [294]:
# merge utility with median on destination

In [295]:
utility = utility.merge(median_inc.astype(float), how='left', left_on='DESTINATION', right_on='CENSUS_TRACT')

In [296]:
utility.rename(columns={'MEDIAN_INC': 'DESTINATION_MEDIAN_INC'}, inplace=True)
utility.drop(columns='CENSUS_TRACT', inplace=True)

In [297]:
# columns for if there is an airport in the origin or destination census tract

In [298]:
airport_census_tract = 21067004207

In [299]:
utility['ORIGIN_AIRPORT'] = utility['ORIGIN'] == airport_census_tract
utility['DESTINATION_AIRPORT'] = utility['DESTINATION'] == airport_census_tract

In [300]:
utility['ORIGIN_AIRPORT'] = utility['ORIGIN_AIRPORT'].astype(int)
utility['DESTINATION_AIRPORT'] = utility['DESTINATION_AIRPORT'].astype(int)

In [301]:
# to convert origin and destination columns to standard notation so we can actually see each census tract
utility['ORIGIN'] = utility['ORIGIN'].map('{:.0f}'.format)
utility['DESTINATION'] = utility['DESTINATION'].map('{:.0f}'.format)

In [302]:
# CALCULATE UTILITY

In [303]:
# calculate private trip utility using: (-0.110)(private travel time in miles) + (-0.138)(private fares in $)

In [304]:
utility.loc[:, 'PRIVATE_UTILITY'] = (-0.110 * utility['PRIVATE_TRAVEL_TIME']) + (-0.138 * utility['PRIVATE_TRIP_FARES'])

In [305]:
# calculate shared trip utility using:
# (-0.573) + (-.110)(shared travel time) + (-0.138)(shared cost) + (-0.068)(median income in origin zone ($50k would be 0.5)) +
# (-0.048)(median income in destination zone ($50k would be 0.5)) - (-2.717)(if origin or destination has airport)

In [306]:
utility.loc[:, 'NT_SHARED_UTILITY'] = (-0.573) + (-.110 * utility['NT_SHARED_TRAVEL_TIME']) + (-0.138 * utility['NT_SHARED_TRIP_FARES']) + (-0.068 * 0.00001 * utility['ORIGIN_MEDIAN_INC']) + (-0.048 * 0.00001 * utility['DESTINATION_MEDIAN_INC']) - (-2.717 * (utility['ORIGIN_AIRPORT'] | utility['DESTINATION_AIRPORT']).astype(int)) 
utility.loc[:, 'AM_SHARED_UTILITY'] = (-0.573) + (-.110 * utility['AM_SHARED_TRAVEL_TIME']) + (-0.138 * utility['AM_SHARED_TRIP_FARES']) + (-0.068 * 0.00001 * utility['ORIGIN_MEDIAN_INC']) + (-0.048 * 0.00001 * utility['DESTINATION_MEDIAN_INC']) - (-2.717 * (utility['ORIGIN_AIRPORT'] | utility['DESTINATION_AIRPORT']).astype(int))
utility.loc[:, 'MD_SHARED_UTILITY'] = (-0.573) + (-.110 * utility['MD_SHARED_TRAVEL_TIME']) + (-0.138 * utility['MD_SHARED_TRIP_FARES']) + (-0.068 * 0.00001 * utility['ORIGIN_MEDIAN_INC']) + (-0.048 * 0.00001 * utility['DESTINATION_MEDIAN_INC']) - (-2.717 * (utility['ORIGIN_AIRPORT'] | utility['DESTINATION_AIRPORT']).astype(int))
utility.loc[:, 'PM_SHARED_UTILITY'] = (-0.573) + (-.110 * utility['PM_SHARED_TRAVEL_TIME']) + (-0.138 * utility['PM_SHARED_TRIP_FARES']) + (-0.068 * 0.00001 * utility['ORIGIN_MEDIAN_INC']) + (-0.048 * 0.00001 * utility['DESTINATION_MEDIAN_INC']) - (-2.717 * (utility['ORIGIN_AIRPORT'] | utility['DESTINATION_AIRPORT']).astype(int))
utility.loc[:, 'EV_SHARED_UTILITY'] = (-0.573) + (-.110 * utility['EV_SHARED_TRAVEL_TIME']) + (-0.138 * utility['EV_SHARED_TRIP_FARES']) + (-0.068 * 0.00001 * utility['ORIGIN_MEDIAN_INC']) + (-0.048 * 0.00001 * utility['DESTINATION_MEDIAN_INC']) - (-2.717 * (utility['ORIGIN_AIRPORT'] | utility['DESTINATION_AIRPORT']).astype(int))

In [307]:
# CALCULATE PROBABILITY OF TAKING A PRIVATE/SHARED RIDE BETWEEN A ZONE PAIR

In [308]:
# equation for private ride probability will be: e^(Uprivate) / (e^(Uprivate) + e^(Ushared))

In [309]:
utility.loc[:, 'PRIV_PROB'] = (np.exp(utility['PRIVATE_UTILITY'])) / ((np.exp(utility['PRIVATE_UTILITY'])) + (np.exp(utility['NT_SHARED_UTILITY'])) + (np.exp(utility['AM_SHARED_UTILITY'])) + (np.exp(utility['MD_SHARED_UTILITY'])) + (np.exp(utility['PM_SHARED_UTILITY'])) + (np.exp(utility['EV_SHARED_UTILITY'])))

In [310]:
# equation for shared ride probability will be: e^(Ushared) / (e^(Ushared) + e^(Uprivate))

In [311]:
utility.loc[:, 'SHARED_PROB'] = ((np.exp(utility['NT_SHARED_UTILITY'])) + (np.exp(utility['AM_SHARED_UTILITY'])) + (np.exp(utility['MD_SHARED_UTILITY'])) + (np.exp(utility['PM_SHARED_UTILITY'])) + (np.exp(utility['EV_SHARED_UTILITY']))) / ((np.exp(utility['PRIVATE_UTILITY'])) + (np.exp(utility['NT_SHARED_UTILITY'])) + (np.exp(utility['AM_SHARED_UTILITY'])) + (np.exp(utility['MD_SHARED_UTILITY'])) + (np.exp(utility['PM_SHARED_UTILITY'])) + (np.exp(utility['EV_SHARED_UTILITY'])))

In [312]:
# probabilities should add up to 1

In [313]:
utility['PROB_SUM'] = utility['PRIV_PROB'] + utility['SHARED_PROB']

In [314]:
# logsum of e^(Uprivate) + e^(Ushared)

In [315]:
utility['LOGSUM'] = np.log((np.exp(utility['NT_SHARED_UTILITY'])) + (np.exp(utility['AM_SHARED_UTILITY'])) + (np.exp(utility['MD_SHARED_UTILITY'])) + (np.exp(utility['PM_SHARED_UTILITY'])) + (np.exp(utility['EV_SHARED_UTILITY'])) + (np.exp(utility['PRIVATE_UTILITY'])))

In [316]:
utility = utility.rename(columns={'DISTANCE_IN_MILES' : 'TRIP_LENGTH_MILES'})

In [317]:
utility = utility[['ORIGIN', 'DESTINATION', 'TRAVEL_TIME_MINUTES', 'TRIP_LENGTH_MILES', 'PRIVATE_TRAVEL_TIME',  'NT_SHARED_TRAVEL_TIME', 'AM_SHARED_TRAVEL_TIME', 'MD_SHARED_TRAVEL_TIME', 'PM_SHARED_TRAVEL_TIME', 'EV_SHARED_TRAVEL_TIME', 'PRIVATE_TRIP_FARES', 'NT_SHARED_TRIP_FARES', 'AM_SHARED_TRIP_FARES', 'MD_SHARED_TRIP_FARES', 'PM_SHARED_TRIP_FARES', 'EV_SHARED_TRIP_FARES', 'PRIVATE_UTILITY', 'NT_SHARED_UTILITY', 'AM_SHARED_UTILITY', 'MD_SHARED_UTILITY', 'PM_SHARED_UTILITY', 'EV_SHARED_UTILITY', 'PRIV_PROB', 'SHARED_PROB', 'PROB_SUM', 'LOGSUM']]

In [318]:
utility.head()

,ORIGIN,DESTINATION,TRAVEL_TIME_MINUTES,TRIP_LENGTH_MILES,PRIVATE_TRAVEL_TIME,NT_SHARED_TRAVEL_TIME,AM_SHARED_TRAVEL_TIME,MD_SHARED_TRAVEL_TIME,PM_SHARED_TRAVEL_TIME,EV_SHARED_TRAVEL_TIME,PRIVATE_TRIP_FARES,NT_SHARED_TRIP_FARES,AM_SHARED_TRIP_FARES,MD_SHARED_TRIP_FARES,PM_SHARED_TRIP_FARES,EV_SHARED_TRIP_FARES,PRIVATE_UTILITY,NT_SHARED_UTILITY,AM_SHARED_UTILITY,MD_SHARED_UTILITY,PM_SHARED_UTILITY,EV_SHARED_UTILITY,PRIV_PROB,SHARED_PROB,PROB_SUM,LOGSUM
0,21067000101,21067000101,0.600000,0.000000,0.600000,0.727506,0.699501,0.659889,0.656997,0.713711,2.077020,2.693250,2.691570,2.689193,2.689020,2.692423,-0.352629,-1.054115,-1.050803,-1.046118,-1.045775,-1.052484,0.286547,0.713453,1.0,0.897224
1,21067000101,21067000102,6.233333,0.644906,6.233333,7.557981,7.267039,6.855511,6.825466,7.414661,4.275116,3.577020,3.559564,3.534872,3.533070,3.568421,-1.275633,-1.927963,-1.893550,-1.844874,-1.841321,-1.911011,0.268560,0.731440,1.0,0.039046
2,21067000101,21067000200,10.383333,0.890833,10.383333,12.589899,12.105254,11.419742,11.369694,12.351160,5.692253,4.059667,4.030589,3.989458,3.986455,4.045343,-1.927698,-2.548198,-2.490874,-2.409792,-2.403872,-2.519960,0.256490,0.743510,1.0,-0.567031
3,21067000101,21067000400,8.950000,1.046286,8.950000,10.851967,10.434224,9.843341,9.800201,10.646185,5.414873,4.069633,4.044569,4.009116,4.006527,4.057286,-1.731753,-2.358548,-2.309137,-2.239247,-2.234145,-2.334208,0.259726,0.740274,1.0,-0.383623
4,21067000101,21067000500,7.533333,1.053952,7.533333,9.134244,8.782624,8.285270,8.248959,8.961035,5.011937,3.972204,3.951107,3.921266,3.919087,3.961811,-1.520314,-2.175860,-2.134271,-2.075443,-2.071149,-2.155373,0.267327,0.732673,1.0,-0.201032


In [279]:
utility.to_csv('E:/CR2/Repos/TNC-Demand-Model-Southeast/Cleaned_Inputs/Intermediate/utiltiy_and_probability_output.csv')